In [16]:
import torch
pwd = '/home/zjy/project/MetaIM/data'
device = torch.device('cuda:4' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=4)

In [17]:
from torch_geometric.datasets import Planetoid

cora_dataset = Planetoid(root=pwd+'/cora', name='cora')
data = cora_dataset[0]
edge_index = data.edge_index

In [18]:
import numpy as np
individual_infection_path = pwd+'/for_meta/cora_individual_infection_sir_100.npy'
seeds_infection_path = pwd+'/for_meta/cora_seed_infection_sir_100.npy'

individual_infection = np.load(individual_infection_path)
seeds_infection = np.load(seeds_infection_path)
individual_infection.shape,seeds_infection.shape

((2708, 2708), (500, 2, 2708))

In [19]:
import torch
from torch_geometric.utils import to_scipy_sparse_matrix
import scipy.sparse as sp

# 转换为 scipy 稀疏矩阵
adj = to_scipy_sparse_matrix(edge_index)


# def normalize_adj(mx):
#     """Row-normalize sparse matrix"""
#     rowsum = np.array(mx.sum(1))
#     r_inv_sqrt = np.power(rowsum, -0.5).flatten()
#     r_inv_sqrt[np.isinf(r_inv_sqrt)] = 0.
#     r_mat_inv_sqrt = sp.diags(r_inv_sqrt)
#     return mx.dot(r_mat_inv_sqrt).transpose().dot(r_mat_inv_sqrt)



# adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
# adj = normalize_adj(adj + sp.eye(adj.shape[0]))
adj = torch.Tensor(adj.toarray()).to_sparse()
adj


tensor(indices=tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
                       [ 633, 1862, 2582,  ...,  598, 1473, 2706]]),
       values=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
       size=(2708, 2708), nnz=10556, layout=torch.sparse_coo)

In [20]:
feat_num = int(seeds_infection[0][0].sum())

In [21]:
from torch.utils.data import Dataset, DataLoader, random_split


class CustomDataset(Dataset):
    def __init__(self, individual_infection,seeds_infection, feat_num):
        self.individual_infection = individual_infection
        self.seeds_infection = seeds_infection
        self.feat_shape = (len(individual_infection), feat_num)

    def __len__(self):
        return len(self.seeds_infection)

    def __getitem__(self, idx):
        seeds= np.nonzero(self.seeds_infection[idx][0])[0]
        
        feature = torch.zeros(self.feat_shape[0],self.feat_shape[1])
        for i in range(len(seeds)):
            seed_i_infection = torch.tensor(self.individual_infection[seeds[i]])
            feature[:, i] = seed_i_infection
            
        label = self.seeds_infection[idx][1]
        
        return feature, label

dataset = CustomDataset(individual_infection,seeds_infection,feat_num)

In [22]:
# 定义划分比例
train_ratio = 0.8
test_ratio = 0.2

# 划分数据集
train_dataset, test_dataset = random_split(dataset, [int(len(dataset)*train_ratio), int(len(dataset)*test_ratio)])

train_batch_size = 64
test_batch_size = 4

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True, drop_last=False)
test_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)

In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv

class GAT(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_heads):
        super(GAT, self).__init__()
        self.conv1 = GATConv(in_channels, hidden_channels, heads=num_heads)
        # self.conv2 = GATConv(hidden_channels * num_heads, hidden_channels * num_heads, heads=1)
        self.conv3 = GATConv(hidden_channels * num_heads, out_channels, heads=1)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.2, training=self.training)
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.2, training=self.training)
        # x = F.elu(self.conv2(x, edge_index))
        
        x = self.conv3(x, edge_index)
        return F.relu(x)

In [24]:
# from data.model.gat import GAT, SpGAT
from torch.optim import Adam, SGD


# forward_model = SpGAT(nfeat=feat_num, 
#                 nhid=64, 
#                 nclass=1, 
#                 dropout=0.2, 
#                 nheads=1, 
#                 alpha=0.2)
forward_model = GAT(feat_num, 256, 1, 1)

optimizer = Adam([{'params': forward_model.parameters()}], 
                 lr=1e-3)

adj = adj.to(device)
forward_model = forward_model.to(device)
forward_model.train()

GAT(
  (conv1): GATConv(135, 256, heads=1)
  (conv3): GATConv(256, 1, heads=1)
)

In [25]:
edge_index = edge_index.to(device)
top_num = 300

for epoch in range(2000):

    total_overall = 0
    forward_loss = 0

    mean_train_accuracy = 0
    for batch_idx, feature_label in enumerate(train_loader):        
        features = feature_label[0].to(device)
        labels = feature_label[1].to(device)
        optimizer.zero_grad()
        
        loss = 0
        train_accuracy = 0
        for i, x_i in enumerate(features):
            y_i = labels[i]
            y_hat = forward_model(x_i, edge_index)
            _, top_indices_true = torch.topk(y_i.clone(), top_num)
            
            _, top_indices_predict = torch.topk(y_hat.clone().squeeze(-1), top_num)
            
            # 将张量数组转换为Python列表
            list1 = top_indices_true.tolist()
            list_pre = top_indices_predict.tolist()

            # 使用集合操作找到交集
            intersection = list(set(list1) & set(list_pre))
            accuracy_i = len(intersection) / top_num       
            train_accuracy += accuracy_i 

            forward_loss = F.mse_loss(y_hat.squeeze(-1), y_i, reduction='sum')        
            loss += forward_loss    
        
        total_overall += loss.item()    
        train_accuracy /= len(features)
        mean_train_accuracy = train_accuracy
        loss.backward()
        optimizer.step()
        # for p in forward_model.parameters():
        #     p.data.clamp_(min=0)
        
        
    print("Epoch: {}".format(epoch+1), 
        "\tTotal: {:.4f}".format(total_overall / train_batch_size),
        "\tMean_train_accuracy: {:.4f}".format(mean_train_accuracy),
        )  
    
    mean_accuracy = 0
    mean_accuracy_sum = 0

    
    for batch_idx, feature_label in enumerate(test_loader):   
        features = feature_label[0].to(device)
        labels = feature_label[1].to(device)
        
        accuracy = 0
        accuracy_sum = 0
        
        for i, x_i in enumerate(features):
            y_i = labels[i]
            _, top_indices_true = torch.topk(y_i, top_num)
            
            y_hat = forward_model(x_i, edge_index)
            
            _, top_indices_predict = torch.topk(y_hat.squeeze(-1), top_num)
            
            sum_pre = torch.sum(x_i, dim=1, keepdim=True)
            _, top_indices_sum = torch.topk(sum_pre.squeeze(-1), top_num)
            
            # 将张量数组转换为Python列表
            list1 = top_indices_true.tolist()
            list_pre = top_indices_predict.tolist()
            
            list_sum = top_indices_sum.tolist()

            # 使用集合操作找到交集
            intersection = list(set(list1) & set(list_pre))
            
            intersection_sum = list(set(list1) & set(list_sum))
            
            accuracy_i = len(intersection) / top_num       
            accuracy += accuracy_i 
            accuracy_sum += len(intersection_sum) / top_num  
        accuracy /= test_batch_size
        accuracy_sum/= test_batch_size
        mean_accuracy = accuracy
        mean_accuracy_sum = accuracy_sum
        break
    
    print(
        "\tMean_test_accuracy: {:.4f}".format(mean_accuracy),
        "\tMean_test_accuracy_sum: {:.4f}".format(mean_accuracy_sum)
        )  

    

Epoch: 1 	Total: 828.4900 	Mean_train_accuracy: 0.3056
	Mean_test_accuracy: 0.2867 	Mean_test_accuracy_sum: 0.3600
Epoch: 2 	Total: 759.7836 	Mean_train_accuracy: 0.3113
	Mean_test_accuracy: 0.2817 	Mean_test_accuracy_sum: 0.3600
Epoch: 3 	Total: 731.7962 	Mean_train_accuracy: 0.3575
	Mean_test_accuracy: 0.3208 	Mean_test_accuracy_sum: 0.3600
Epoch: 4 	Total: 708.8499 	Mean_train_accuracy: 0.3312
	Mean_test_accuracy: 0.3358 	Mean_test_accuracy_sum: 0.3600
Epoch: 5 	Total: 692.7156 	Mean_train_accuracy: 0.3585
	Mean_test_accuracy: 0.3675 	Mean_test_accuracy_sum: 0.3600
Epoch: 6 	Total: 678.1318 	Mean_train_accuracy: 0.3837
	Mean_test_accuracy: 0.3925 	Mean_test_accuracy_sum: 0.3600
Epoch: 7 	Total: 666.0510 	Mean_train_accuracy: 0.3831
	Mean_test_accuracy: 0.3775 	Mean_test_accuracy_sum: 0.3600
Epoch: 8 	Total: 654.7167 	Mean_train_accuracy: 0.4088
	Mean_test_accuracy: 0.4242 	Mean_test_accuracy_sum: 0.3600
Epoch: 9 	Total: 644.5070 	Mean_train_accuracy: 0.4231
	Mean_test_accuracy: 0.42